In [37]:
import json
with open("resnet50_layer3_feature_descriptor_latentSemantics.json","r") as f:
    LDA_resnet_latent_semantics = json.loads(f.read())

In [47]:
LDA_resnet_latent_semantics["latent_semantics"]

{'0': '0.027*"0.07" + 0.026*"0.04" + 0.026*"0.03" + 0.025*"0.01" + 0.023*"0.02" + 0.023*"0.05" + 0.022*"0.06" + 0.021*"0.1" + 0.019*"0.15" + 0.018*"0.09" + 0.018*"0.19" + 0.017*"0.08" + 0.017*"0.24" + 0.017*"0.14" + 0.016*"0.17" + 0.016*"0.2" + 0.016*"0.11" + 0.016*"0.12" + 0.016*"0.21" + 0.015*"0.13" + 0.015*"0.18" + 0.014*"0.26" + 0.014*"0.16" + 0.014*"0.28" + 0.014*"0.0" + 0.013*"0.3" + 0.012*"0.31" + 0.012*"0.29" + 0.012*"0.22" + 0.011*"0.25" + 0.011*"0.37" + 0.010*"0.35" + 0.010*"0.32" + 0.010*"0.23" + 0.010*"0.42" + 0.010*"0.38" + 0.009*"0.41" + 0.009*"0.36" + 0.009*"0.48" + 0.009*"0.47" + 0.009*"0.4" + 0.009*"0.39" + 0.008*"0.44" + 0.008*"0.34" + 0.008*"0.27" + 0.008*"0.55" + 0.007*"0.61" + 0.007*"0.5" + 0.007*"0.49" + 0.007*"0.45" + 0.007*"0.53" + 0.006*"0.43" + 0.006*"0.57" + 0.006*"0.33" + 0.006*"0.51" + 0.006*"0.63" + 0.006*"0.54" + 0.006*"0.46" + 0.006*"0.62" + 0.006*"0.52" + 0.005*"0.64" + 0.005*"0.6" + 0.005*"0.59" + 0.005*"0.56" + 0.005*"0.66" + 0.005*"0.69" + 0.005*"0.6

In [79]:

for j in range(10):
    sum = 0
    for i in LDA_resnet_latent_semantics["latent_semantics"][str(j)].split(' + ')[:10]:
        temp = i.split('*')
        sum+=float(temp[0])
    print(sum)

0.22999999999999995
0.21799999999999994
0.25899999999999995
0.23499999999999996
0.24699999999999994
0.23299999999999996
0.23399999999999996
0.26099999999999995
0.21599999999999997
0.23499999999999996


In [42]:
LDA_resnet_latent_semantics["image_weight_matrix"][1]

[[6, '0.16037217'],
 [8, '0.12580548'],
 [1, '0.11706544'],
 [7, '0.1128946'],
 [9, '0.10750493'],
 [5, '0.09865117'],
 [3, '0.095427155'],
 [0, '0.089782365'],
 [4, '0.052630633'],
 [2, '0.039866008']]

In [43]:
import torch
import torchvision
from torchvision.io import read_image
import torchvision.datasets
dataset_path = "/Users/lalitarvind/Downloads/MWD_Team_project"
dataset = torchvision.datasets.Caltech101(root = dataset_path,download = False)
category_names = dataset.annotation_categories
ending_index = dict()
count = 0
for i in range(0,len(dataset),2):
    image,label = dataset[i]
    if label in ending_index:
        ending_index[label]["end_idx"] = int(i/2)
    else:
        ending_index[label] = {"start_idx":int(i/2),"end_idx":int(i/2)}

In [31]:
ending_index

{0: {'start_idx': 0, 'end_idx': 217},
 1: {'start_idx': 218, 'end_idx': 434},
 2: {'start_idx': 435, 'end_idx': 534},
 3: {'start_idx': 535, 'end_idx': 933},
 4: {'start_idx': 934, 'end_idx': 961},
 5: {'start_idx': 962, 'end_idx': 1361},
 6: {'start_idx': 1362, 'end_idx': 1382},
 7: {'start_idx': 1383, 'end_idx': 1403},
 8: {'start_idx': 1404, 'end_idx': 1426},
 9: {'start_idx': 1427, 'end_idx': 1453},
 10: {'start_idx': 1454, 'end_idx': 1476},
 11: {'start_idx': 1477, 'end_idx': 1493},
 12: {'start_idx': 1494, 'end_idx': 1557},
 13: {'start_idx': 1558, 'end_idx': 1606},
 14: {'start_idx': 1607, 'end_idx': 1627},
 15: {'start_idx': 1628, 'end_idx': 1670},
 16: {'start_idx': 1671, 'end_idx': 1715},
 17: {'start_idx': 1716, 'end_idx': 1740},
 18: {'start_idx': 1741, 'end_idx': 1762},
 19: {'start_idx': 1763, 'end_idx': 1823},
 20: {'start_idx': 1824, 'end_idx': 1847},
 21: {'start_idx': 1848, 'end_idx': 1876},
 22: {'start_idx': 1877, 'end_idx': 1907},
 23: {'start_idx': 1908, 'end_idx'

In [35]:
import numpy as np
representatives = dict()#[mean_value(topic0), mean_value(topic1),...]
for label,indices in ending_index.items():
    label_topic_distribution = dict() # {0:[topic0(img1), topic0(img2),...],1:[topic1(img1),topic1(img2),...]}}
    for item in LDA_resnet_latent_semantics["image_weight_matrix"][indices["start_idx"]:indices["end_idx"]]:
        for lst in item:
            if lst[0] in label_topic_distribution.keys():
                label_topic_distribution[lst[0]].append(float(lst[1]))
            else:
                label_topic_distribution[lst[0]] = [float(lst[1])]
    label_topic_distribution = dict(sorted(label_topic_distribution.items()))
    print(label_topic_distribution)
    representative = []
    for topic,lst in label_topic_distribution.items():
        representative.append(np.mean(lst))
    representatives[label] = representative
print("representatives",representatives)

{0: [0.098489255, 0.049518853, 0.09110708, 0.08997585, 0.102383666, 0.12102148, 0.11580785, 0.07650853, 0.13246334, 0.11984439, 0.042695574, 0.05103257, 0.1036049, 0.07940652, 0.15206662, 0.14564407, 0.065744154, 0.06366354, 0.122724734, 0.14000885, 0.13198088, 0.10746293, 0.106467046, 0.08481159, 0.11668366, 0.06832015, 0.06542161, 0.08274948, 0.10079109, 0.113783784, 0.07442007, 0.11448446, 0.10479383, 0.128009, 0.10714202, 0.09891243, 0.08496426, 0.10502771, 0.1017814, 0.102128915, 0.05877082, 0.04197804, 0.14398792, 0.09199748, 0.1449563, 0.051446024, 0.047362674, 0.06382412, 0.06828466, 0.09378699, 0.0656824, 0.13579056, 0.05739829, 0.13762085, 0.078174934, 0.14056669, 0.07539898, 0.113840945, 0.112793565, 0.07322351, 0.11944816, 0.06162945, 0.08821515, 0.11988235, 0.10952303, 0.06549858, 0.08968095, 0.05741849, 0.1045181, 0.09268045, 0.10275234, 0.08045085, 0.08638191, 0.12723279, 0.081436075, 0.09284028, 0.06886136, 0.103492625, 0.09361771, 0.09586129, 0.09999351, 0.085577235, 0

In [84]:
print(representatives[0])

[0.0953171081889401, 0.0980858959078341, 0.11193087304608294, 0.09805769771889401, 0.11284263203686636, 0.09859125041935485, 0.08502047544009218, 0.10442677771889401, 0.09675655033179724, 0.09880304639631335]


In [ ]:
import sys
def preprocess(feature_space,f1):
    if(feature_space in ('hog_feature_descriptor')):
        f1=np.array(f1).reshape((1,900)).flatten()
        min_val=f1.min()
        if(min_val<0):
            f1=f1-min_val
        max_val=f1.max()
        f1=f1/max_val
    
    #normalizing Colormoments feature values
    elif(feature_space in ('color_moments_feature_descriptor')):
        #extracting minimum value for each mean, SD, Skewness
        min_vals=[sys.maxsize]*3
        for j in f1:#iterating through gird_rows
            for k in j:#iterating through grid_columns
                for l in k:#iterating through channels
                    for idx,ele in enumerate(l):#iterating through 3 color_moments(mean,SD,Skewness)
                        if(min_vals[idx]>ele):
                            min_vals[idx]=ele

        #identifying which color_moment (mean/SD/Skewness) have negative minimum values
        idxes_to_be_added=[]
        for j in range(3):
            if(min_vals[j]<0):
                idxes_to_be_added.append(j)

        #adding -(negative min value of each color_moment found) to all values belonging to that color moment to help normalize the values 
        for j in f1:#iterating through grid_rows
            for k in j:#iterating through grid_columns
                for l in k:#iterating through channels
                    for idx in idxes_to_be_added:#iterating through 3 color_moments(mean,SD,Skewness)
                        l[idx]+=-(min_vals[idx])

        min_vals=[sys.maxsize]*3
        max_vals=[-sys.maxsize+1]*3
        #identifying the maximum value of each color_moment
        for j in f1:#iterating through grid_rows
            for k in j:#iterating through grid_columns
                for l in k:#iterating through channels
                    for idx,ele in enumerate(l):#iterating through 3 color_moments(mean,SD,Skewness)
                        if(max_vals[idx]<ele):
                            max_vals[idx]=ele
                                
        #divinding by maximum value to normalize each color_moment
        for j in f1:#iterating through grid_rows
            for k in j:#iterating through grid_columns
                for l in k:#iterating through channels
                    for idx in range(len(l)):#iterating through 3 color_moments(mean,SD,Skewness)
                        l[idx]/=max_vals[idx]
        f1=np.array(f1).reshape((1,900)).flatten()

    return f1

In [ ]:
# def getkey(i,id2num):
#     for j,k in id2num.items():
#         if k == i:
#             return j
        
# def replace_ids(f1, id2num):
#     temp_dict = dict()
#     for i in f1:
#         id = getkey(round(i,2),id2num)#rounding off each value of the selected feature descriptor to make them discrete
#         if id not in temp_dict.keys():
#             temp_dict[id] = 1
#         else:
#             temp_dict[id]+=1
#     return temp_dict

In [71]:
def calculate_LDA_latentSemantics(query_features,latent_semantics):
    corpus = dict()
    for i in query_features:
        temp = round(i,2)
        if temp not in corpus.keys():
            corpus[temp] = 1
        else:
            corpus[temp]+=1
    
    query_latent_semantics = dict()
    for i in latent_semantics.keys():
        feature_weight = dict()
        lst = latent_semantics[i].split(" + ")[:30]
        for ele in lst:
            temp = ele.split("*")
            feature_weight[float(temp[1].replace('"',""))] = float(temp[0])
        weight = 0
        for val in corpus.keys():
            f_weight = 0
            if val in feature_weight:
                f_weight = feature_weight[val]
            weight+= f_weight
        query_latent_semantics[i] = weight
    return query_latent_semantics

In [72]:
from pymongo import MongoClient
#Connecting to mongoDB client and creating database and collections
client = MongoClient("localhost", 27017)
DB = client.caltech101db
collection = DB.caltech101withGSimages
features = collection.find_one({"image_id":2344})["resnet50_fc_feature_descriptor"]
query_latent_semantics = calculate_LDA_latentSemantics(features, LDA_resnet_latent_semantics["latent_semantics"])
query_latent_semantics

{'0': 0.5250000000000001,
 '1': 0.5290000000000001,
 '2': 0.5590000000000002,
 '3': 0.5400000000000001,
 '4': 0.5530000000000003,
 '5': 0.5420000000000003,
 '6': 0.5310000000000002,
 '7': 0.5500000000000002,
 '8': 0.5210000000000002,
 '9': 0.5400000000000001}

In [80]:
len(representatives)

101

In [88]:
type(representatives[0])

list

In [117]:
lst = LDA_resnet_latent_semantics["image_weight_matrix"][1250]
lst =  sorted(lst,key = lambda a:a[0])
lst

[[0, '0.108620696'],
 [1, '0.10147213'],
 [2, '0.083024815'],
 [3, '0.114580244'],
 [4, '0.1406856'],
 [5, '0.0851058'],
 [6, '0.08833181'],
 [7, '0.15085468'],
 [8, '0.06880816'],
 [9, '0.058516026']]

In [108]:
category_names[0]

'Faces_2'

In [118]:
query = lst
distances = []
for label,rep in representatives.items():
    temp = 0
    for i in range(len(rep)):
        temp = (rep[i]-float(query[i][1]))**2
    distances.append((label,category_names[label],temp**0.5))
distances = sorted(distances,key = lambda a:a[2],reverse=True)

In [119]:
distances[:10]

[(45, 'gramophone', 0.06541657592),
 (69, 'pagoda', 0.06330053677272728),
 (24, 'cougar_body', 0.062265101954545446),
 (15, 'buddha', 0.060630269571428586),
 (68, 'okapi', 0.056834616894736845),
 (29, 'crocodile_head', 0.05637642152000001),
 (96, 'wheelchair', 0.05521285599999999),
 (54, 'kangaroo', 0.054964028714285726),
 (97, 'wild_cat', 0.05488892337500001),
 (50, 'helicopter', 0.05344110569767441)]